In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## vit_model

In [2]:
"""
original code from rwightman:
https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
"""
from functools import partial
from collections import OrderedDict

import torch
import torch.nn as nn


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(nn.Module):
    """
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class PatchEmbed(nn.Module):
    """
    2D Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_c=256, embed_dim=768, norm_layer=None):
        super().__init__()
        img_size = (img_size, img_size)
        patch_size = (patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]

        self.proj = nn.Conv2d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."

        # flatten: [B, C, H, W] -> [B, C, HW]
        # transpose: [B, C, HW] -> [B, HW, C]
        x = self.proj(x).flatten(2).transpose(1, 2)
        x = self.norm(x)
        return x


class Attention(nn.Module):
    def __init__(self,
                 dim,   # dim of token
                 num_heads=8,
                 qkv_bias=False,
                 qk_scale=None,
                 attn_drop_ratio=0.,
                 proj_drop_ratio=0.):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop_ratio)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop_ratio)

    def forward(self, x):
        # [batch_size, num_patches + 1, total_embed_dim]
        B, N, C = x.shape

        # qkv(): -> [batch_size, num_patches + 1, 3 * total_embed_dim]
        # reshape: -> [batch_size, num_patches + 1, 3, num_heads, embed_dim_per_head]
        # permute: -> [3, batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        # transpose: -> [batch_size, num_heads, embed_dim_per_head, num_patches + 1]
        # @: multiply -> [batch_size, num_heads, num_patches + 1, num_patches + 1]
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        # @: multiply -> [batch_size, num_heads, num_patches + 1, embed_dim_per_head]
        # transpose: -> [batch_size, num_patches + 1, num_heads, embed_dim_per_head]
        # reshape: -> [batch_size, num_patches + 1, total_embed_dim]
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Mlp(nn.Module):
    """
    MLP as used in Vision Transformer, MLP-Mixer and related networks
    """
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class Block(nn.Module):
    def __init__(self,
                 dim,
                 num_heads,
                 mlp_ratio=4.,
                 qkv_bias=False,
                 qk_scale=None,
                 drop_ratio=0.,
                 attn_drop_ratio=0.,
                 drop_path_ratio=0.,
                 act_layer=nn.GELU,
                 norm_layer=nn.LayerNorm):
        super(Block, self).__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                              attn_drop_ratio=attn_drop_ratio, proj_drop_ratio=drop_ratio)
        # NOTE: drop path for stochastic depth, we shall see if this is better than dropout here
        self.drop_path = DropPath(drop_path_ratio) if drop_path_ratio > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop_ratio)

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_c=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, representation_size=None, distilled=False, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None):
        """
        Args:
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_c (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (int): embedding dimension
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            distilled (bool): model includes a distillation token and head as in DeiT models
            drop_ratio (float): dropout rate
            attn_drop_ratio (float): attention dropout rate
            drop_path_ratio (float): stochastic depth rate
            embed_layer (nn.Module): patch embedding layer
            norm_layer: (nn.Module): normalization layer
        """
        super(VisionTransformer, self).__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_tokens = 2 if distilled else 1
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer = act_layer or nn.GELU

        self.patch_embed = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + self.num_tokens, embed_dim))
        self.pos_drop = nn.Dropout(p=drop_ratio)

        dpr = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # stochastic depth decay rule
        self.blocks = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr[i],
                  norm_layer=norm_layer, act_layer=act_layer)
            for i in range(depth)
        ])
        self.norm = norm_layer(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ]))
        else:
            self.has_logits = False
            self.pre_logits = nn.Identity()

        # Classifier head(s)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim, self.num_classes) if num_classes > 0 else nn.Identity()

        # Weight init
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        if self.dist_token is not None:
            nn.init.trunc_normal_(self.dist_token, std=0.02)

        nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.apply(_init_vit_weights)

    def forward_features(self, x):
        # [B, C, H, W] -> [B, num_patches, embed_dim]
        x = self.patch_embed(x)  # [B, 196, 768]
        # [1, 1, 768] -> [B, 1, 768]
        cls_token = self.cls_token.expand(x.shape[0], -1, -1)
        if self.dist_token is None:
            x = torch.cat((cls_token, x), dim=1)  # [B, 197, 768]
        else:
            x = torch.cat((cls_token, self.dist_token.expand(x.shape[0], -1, -1), x), dim=1)

        x = self.pos_drop(x + self.pos_embed)  # [B, 197, 768]
        x = self.blocks(x)
        x = self.norm(x)
        if self.dist_token is None:
            return self.pre_logits(x[:, 0])  # [B, 768]
        else:
            return x[:, 0], x[:, 1]

    def forward(self, x):
        x = self.forward_features(x)  # [B, 768]
        if self.head_dist is not None:
            x, x_dist = self.head(x[0]), self.head_dist(x[1])
            if self.training and not torch.jit.is_scripting():
                # during inference, return the average of both classifier predictions
                return x, x_dist
            else:
                return (x + x_dist) / 2
        else:
            x = self.head(x)
        return x


class VisionTransformer_double_branch(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_c=3, num_classes=1000,
                 embed_dim=768, depth=12, num_heads=12, mlp_ratio=4.0, qkv_bias=True,
                 qk_scale=None, representation_size=None, distilled=False, drop_ratio=0.,
                 attn_drop_ratio=0., drop_path_ratio=0., embed_layer=PatchEmbed, norm_layer=None,
                 act_layer=None):
        """
        double_branch means that transformer will be used to generate output for 2 input. Then the 2 result from
            transformer structure will be concat for final output
        Args:
            img_size (int, tuple): input image size
            patch_size (int, tuple): patch size
            in_c (int): number of input channels
            num_classes (int): number of classes for classification head
            embed_dim (int): embedding dimension
            depth (int): depth of transformer
            num_heads (int): number of attention heads
            mlp_ratio (int): ratio of mlp hidden dim to embedding dim
            qkv_bias (bool): enable bias for qkv if True
            qk_scale (float): override default qk scale of head_dim ** -0.5 if set
            representation_size (Optional[int]): enable and set representation layer (pre-logits) to this value if set
            distilled (bool): model includes a distillation token and head as in DeiT models
            drop_ratio (float): dropout rate
            attn_drop_ratio (float): attention dropout rate
            drop_path_ratio (float): stochastic depth rate
            embed_layer (nn.Module): patch embedding layer
            norm_layer: (nn.Module): normalization layer
        """
        super(VisionTransformer_double_branch, self).__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.num_tokens = 2 if distilled else 1

        # For branch_1
        norm_layer_1 = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer_1 = act_layer or nn.GELU

        self.patch_embed_1 = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        num_patches_1 = self.patch_embed_1.num_patches

        self.cls_token_1 = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token_1 = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        self.pos_embed_1 = nn.Parameter(torch.zeros(1, num_patches_1 + self.num_tokens, embed_dim))
        self.pos_drop_1 = nn.Dropout(p=drop_ratio)

        dpr_1 = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # stochastic depth decay rule
        self.blocks_1 = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr_1[i],
                  norm_layer=norm_layer_1, act_layer=act_layer_1)
            for i in range(depth)
        ])
        self.norm_1 = norm_layer_1(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits_1 = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ]))
        else:
            self.has_logits = False
            self.pre_logits_1 = nn.Identity()

        # Weight init
        nn.init.trunc_normal_(self.pos_embed_1, std=0.02)
        if self.dist_token_1 is not None:
            nn.init.trunc_normal_(self.dist_token_1, std=0.02)

        nn.init.trunc_normal_(self.cls_token_1, std=0.02)
        self.apply(_init_vit_weights)

        # For branch_2
        norm_layer_2 = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        act_layer_2 = act_layer or nn.GELU

        self.patch_embed_2 = embed_layer(img_size=img_size, patch_size=patch_size, in_c=in_c, embed_dim=embed_dim)
        num_patches_2 = self.patch_embed_2.num_patches

        self.cls_token_2 = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.dist_token_2 = nn.Parameter(torch.zeros(1, 1, embed_dim)) if distilled else None
        self.pos_embed_2 = nn.Parameter(torch.zeros(1, num_patches_2 + self.num_tokens, embed_dim))
        self.pos_drop_2 = nn.Dropout(p=drop_ratio)

        dpr_2 = [x.item() for x in torch.linspace(0, drop_path_ratio, depth)]  # stochastic depth decay rule
        self.blocks_2 = nn.Sequential(*[
            Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                  drop_ratio=drop_ratio, attn_drop_ratio=attn_drop_ratio, drop_path_ratio=dpr_2[i],
                  norm_layer=norm_layer_2, act_layer=act_layer_2)
            for i in range(depth)
        ])
        self.norm_2 = norm_layer_2(embed_dim)

        # Representation layer
        if representation_size and not distilled:
            self.has_logits = True
            self.num_features = representation_size
            self.pre_logits_2 = nn.Sequential(OrderedDict([
                ("fc", nn.Linear(embed_dim, representation_size)),
                ("act", nn.Tanh())
            ]))
        else:
            self.has_logits = False
            self.pre_logits_2 = nn.Identity()

        # Weight init
        nn.init.trunc_normal_(self.pos_embed_2, std=0.02)
        if self.dist_token_1 is not None:
            nn.init.trunc_normal_(self.dist_token_2, std=0.02)

        nn.init.trunc_normal_(self.cls_token_2, std=0.02)
        self.apply(_init_vit_weights)

        # Classifier head(s) for double branch network
        self.head = nn.Linear(self.num_features * 2, num_classes) if num_classes > 0 else nn.Identity()
        self.head_dist = None
        if distilled:
            self.head_dist = nn.Linear(self.embed_dim * 2, self.num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x, patch_embed, cls_token, dist_token,
                         pos_embed, pos_drop,
                         blocks, norm, pre_logits):
        # [B, C, H, W] -> [B, num_patches, embed_dim]
        x = patch_embed(x)  # [B, 196, 768]
        # [1, 1, 768] -> [B, 1, 768]
        cls_token = cls_token.expand(x.shape[0], -1, -1)
        if dist_token is None:
            x = torch.cat((cls_token, x), dim=1)  # [B, 197, 768]
        else:
            x = torch.cat((cls_token, dist_token.expand(x.shape[0], -1, -1), x), dim=1)

        x = pos_drop(x + pos_embed)
        x = blocks(x)
        x = norm(x)
        if dist_token is None:
            return pre_logits(x[:, 0])
        else:
            return x[:, 0], x[:, 1]

    def forward(self, x_1, x_2):
        # concat result of x_1 and x_2 and do projection for final output
        x_1 = self.forward_features(x_1, self.patch_embed_1, self.cls_token_1, self.dist_token_1,
                                    self.pos_embed_1, self.pos_drop_1,
                                    self.blocks_1, self.norm_1, self.pre_logits_1)  # [B, 768]
        x_2 = self.forward_features(x_2, self.patch_embed_2, self.cls_token_2, self.dist_token_2,
                                    self.pos_embed_2, self.pos_drop_2,
                                    self.blocks_2, self.norm_2, self.pre_logits_2)  # [B, 768]
        if self.head_dist is not None:
            x, x_dist = self.head(torch.cat((x_1[0], x_2[0]), dim=1)), \
                        self.head_dist(torch.cat((x_1[1], x_2[1]), dim=1))
            if self.training and not torch.jit.is_scripting():
                # during inference, return the average of both classifier predictions
                return x, x_dist
            else:
                return (x + x_dist) / 2
        else:
            x = torch.cat((x_1, x_2), dim=1)
            x = self.head(x)

        return x


def _init_vit_weights(m):
    """
    ViT weight initialization
    :param m: module
    """
    if isinstance(m, nn.Linear):
        nn.init.trunc_normal_(m.weight, std=.01)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode="fan_out")
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm):
        nn.init.zeros_(m.bias)
        nn.init.ones_(m.weight)


def vit_base_patch16_224_in21k_double_branch(img_size=224, num_classes: int = 21843, in_c=3, has_logits: bool = True):
    """
    ViT-Base model (ViT-B/16) from original paper (https://arxiv.org/abs/2010.11929).
    ImageNet-21k weights @ 224x224, source https://github.com/google-research/vision_transformer.
    weights ported from official Google JAX impl:
    https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_patch16_224_in21k-e5005f0a.pth
    """
    model = VisionTransformer_double_branch(img_size=img_size,
                                            patch_size=16,
                                            in_c=in_c,
                                            embed_dim=768,
                                            depth=12,
                                            num_heads=12,
                                            representation_size=768 if has_logits else None,
                                            num_classes=num_classes)
    return model



## utils

In [3]:
!pip install logzero

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
from logzero import logger
import sys
import json
import pickle
import random

import numpy as np

import torch
from torch import nn
from tqdm import tqdm


def train_one_epoch(model, optimizer, data_loader, device, epoch):
    logger.info(f"Epoch {epoch + 1}")
    model.train()
    loss_function = nn.MSELoss()
    # train_loss is the accumulate loss across steps in each epoch
    train_loss = 0.0
    # train_acc_all_labels is used to calculate the avg performance of all labels
    # train_acc_all_labels = 0.0
    # train_acc_separate_labels is used to calculate the performance of all labels separately
    # total number of labels = len(train_loader.dataset.labels[0]) labels
    # train_acc_separate_labels = torch.tensor([0 for _ in range(len(data_loader.dataset.labels[0]))]).float()
    # train_acc_separate_labels = train_acc_separate_labels.to(device)
    optimizer.zero_grad()

    sample_num = 0
    data_loader = tqdm(data_loader)
    for step, data in enumerate(data_loader):

        images, labels = data
        images = images.to(device)  # images is [B, num_data, img_length, img_height, channel] for multi input
        labels = labels.to(device)
        sample_num += images.shape[0]

        pred = model(images[:, 0, :, :], images[:, 1, :, :])  # for double branch

        # train_acc_all_labels += abs((pred - labels).sum().item()) / labels.size()[1]
        # train_acc_separate_labels += torch.eq(pred, labels).float().sum(dim=0)

        loss = loss_function(pred, labels)
        loss.backward()
        train_loss += loss.item()

        data_loader.desc = "[train epoch {}] mse: {:.3f}".\
            format(epoch + 1,
                   train_loss / (step + 1))

        if not torch.isfinite(loss):
            print('WARNING: non-finite loss, ending training ', loss)
            sys.exit(1)

        optimizer.step()
        optimizer.zero_grad()

    return train_loss / (step + 1)  # train_acc_all_labels / sample_num  train_acc_separate_labels / sample_num


@torch.no_grad()
def evaluate(model, data_loader, device, epoch):
    loss_function = nn.MSELoss()

    model.eval()

    test_loss = 0.0
    # test_acc_all_labels = 0.0
    # test_acc_separate_labels = torch.tensor([0 for _ in range(len(data_loader.dataset.labels[0]))]).float()
    # test_acc_separate_labels = test_acc_separate_labels.to(device)

    sample_num = 0
    data_loader = tqdm(data_loader)
    predict_y_list = list()
    test_labels_list = list()
    for step, data in enumerate(data_loader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        sample_num += images.shape[0]

        pred = model(images[:, 0, :, :], images[:, 1, :, :])
        # test_acc_all_labels += (pred - labels).sum().item() / labels.size()[1]
        # test_acc_separate_labels += torch.eq(pred_class, labels).float().sum(dim=0)
        predict_y_list.extend(list(np.array(pred.cpu())))
        test_labels_list.extend(list(np.array(labels.cpu())))
        loss = loss_function(pred, labels)
        test_loss += loss

        data_loader.desc = "[test epoch {}] mse: {:.3f}".\
            format(epoch + 1,
                   test_loss / (step + 1))

    return test_loss / (step + 1), predict_y_list, test_labels_list


## my_dataset

In [5]:
import os

import numpy as np
import torch


def data_loader(path):
    # numpy reader
    numpy_data = np.load(path)
    # transfer the numpy data info float numpy before transferred to tensor, other wise model will break down
    numpy_data = numpy_data.astype(np.float32)
    return numpy_data


# customize data reader
class MyDataSet(torch.nn.Module):
    def __init__(self, data_file_path, csv_file_path, num_data=1, transform=None, loader=data_loader):
        super(MyDataSet, self).__init__()
        # open the csv file that save the data route and labels
        fp = open(csv_file_path, 'r')
        fp.readline()
        data_route_info = []
        labels_info = []
        # store data and label
        for line in fp:
            line = line.strip('\n')
            information = line.split(',')
            data_route_info.append(information[:num_data])
            # transfer label from str to float
            labels_info.append([float(i) for i in information[num_data:len(information)]])
        self.num_data = num_data
        self.data_path = data_file_path
        self.data = data_route_info
        self.labels = labels_info
        self.transform = transform
        self.loader = loader

    # rewrite this function to customize reading data
    def __getitem__(self, item):
        # get the data name and data label
        dataName = self.data[item]
        label = self.labels[item]
        if self.num_data > 1:
            # read the data
            data = []
            for name in dataName:
                data_subset = self.loader(os.path.join(self.data_path, name))
                # parse the data
                if self.transform is not None:
                    data_subset = self.transform(data_subset)
                data.append(data_subset.unsqueeze_(0))
            data = torch.cat(data, dim=0)
        else:
            data = self.loader(os.path.join(self.data_path, dataName))
            if self.transform is not None:
                data = self.transform(data)
        # transfer to float tensor since loss in nn just accepts float
        label = torch.FloatTensor(label)
        return data, label

    # rewrite the function to show how much data points are in the data loader
    def __len__(self):
        return len(self.data)

    @staticmethod
    def collate_fn(batch):
        """
        # for achieving default_collate officially please refer to:
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images, labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        return images, labels
        """


## predict

In [6]:
import os
import gc

import numpy as np
import torch
from torchvision import transforms
import matplotlib.pyplot as plt

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.cuda.get_device_name()

'Tesla T4'

In [9]:
# load the data
data_root = "/content/drive/MyDrive/Project/Pore Flow/data"  # get data root path
data_path = os.path.join(data_root, "data_model")  # data set path
csv_path = os.path.join(data_root, "data_route")
assert os.path.exists(data_path), "{} path does not exist.".format(data_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
data_transform = transforms.Compose([
        # transforms.Resize(224),
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])


# create data set
train_dataset = MyDataSet(data_file_path=os.path.join(data_path),
              csv_file_path=os.path.join(csv_path, "split_by_sample_mpa_elec", "train", "data_model_train.csv"),
              num_data=2,
              transform=data_transform)

test_dataset = MyDataSet(data_file_path=os.path.join(data_path),
              csv_file_path=os.path.join(csv_path, "split_by_sample_mpa_elec", "test", "data_model_test.csv"),
              num_data=2,
              transform=data_transform)

In [11]:
# create model
model = vit_base_patch16_224_in21k_double_branch(img_size=256, num_classes=1, in_c=256, has_logits=False).to(device)

# hyper param
model_weight_path = r"/content/drive/MyDrive/Project/Pore Flow/result/model_trained/ViT_double_branch/36-65/"
state = torch.load(os.path.join(model_weight_path, "ViT-16.pth"))
model.load_state_dict(state)
model.to(device)
model.eval()
batch_size = 2
save_path = "/content/drive/MyDrive/Project/Pore Flow/result/model_trained/ViT_double_branch"

train_data_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          pin_memory=True,
                          num_workers=1,  # set to 1 for now
                          # collate_fn=train_dataset.collate_fn
                          )

test_data_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          pin_memory=True,
                          num_workers=1,  # set to 1 for now
                          # collate_fn=train_dataset.collate_fn
                          )

with torch.no_grad():
    torch.cuda.empty_cache()
    print(gc.collect())
    test_loss, predict_y_list, true_y_list = evaluate(model=model,
                               data_loader=train_data_loader,
                               device=device,
                               epoch=0)
    np.save(file=os.path.join(save_path, "pred_train_data.npy"), arr=np.array(predict_y_list))
    np.save(file=os.path.join(save_path, "true_train_data.npy"), arr=np.array(true_y_list))

with torch.no_grad():
    torch.cuda.empty_cache()
    print(gc.collect())
    test_loss, predict_y_list, true_y_list = evaluate(model=model,
                               data_loader=test_data_loader,
                               device=device,
                               epoch=1)
    np.save(file=os.path.join(save_path, "pred_test_data.npy"), arr=np.array(predict_y_list))
    np.save(file=os.path.join(save_path, "true_test_data.npy"), arr=np.array(true_y_list))

9


[test epoch 1] mse: 0.003: 100%|██████████| 174/174 [14:39<00:00,  5.06s/it]


0


[test epoch 2] mse: 0.019: 100%|██████████| 123/123 [10:07<00:00,  4.94s/it]
